In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.ons.gov.uk/businessindustryandtrade/internationaltrade/datasets/internationaltradeinservicesreferencetables')
    tabs = { tab.name: tab for tab in scraper.distribution().as_databaker() }

In [2]:
tab = tabs['Table C5 2009-2012']

In [3]:
observations = tab.excel_ref('E6').expand(DOWN).expand(RIGHT).is_not_blank()

In [4]:
Year = tab.excel_ref('E4').expand(RIGHT).is_not_whitespace()

In [5]:
Flow = tab.excel_ref('E3').expand(RIGHT).is_not_blank()

In [6]:
geo1 = tab.excel_ref('A5').expand(DOWN).is_not_blank() - tab.excel_ref('A94').expand(DOWN)
geo1

{<A78 'Technical services'>, <A93 '..  Denotes disclosive data'>, <A85 'Other Trade  in Services'>, <A41 'Construction Services'>, <A56 'Merchanting and Other Trade related Services'>, <A92 '-   Denotes nil or less than £500,000'>, <A5 'Agricultural, Mining and On-site Processing Services '>, <A70 'Royalties and Licenses'>, <A62 'Personal, Cultural and Recreational  Services'>, <A13 'Business and Professional Services'>, <A31 'Communications and Computer Information Services'>, <A51 'Insurance Services'>, <A47 'Financial Services'>, <A89 'Total International Trade in Services'>}

In [7]:
geo3 = tab.excel_ref('C5').expand(DOWN).is_not_blank() - tab.excel_ref('A94').expand(DOWN)
geo3

{<C8 'Waste treatment and depollution'>, <C16 'Management consulting'>, <C24 'Property management'>, <C9 'Other on-site processing services'>, <C63 'Audio- Visual and related services '>, <C80 'Engineering'>, <C35 'News agency services'>, <C58 'Other trade - related services'>, <C26 'Services between related enterprises'>, <C52 'Insurance Claims'>, <C15 'Advertising'>, <C57 'Merchanting'>, <C33 'Telecommunications'>, <C14 'Accountancy, auditing, bookkeeping and tax consulting services'>, <C17 'Public relations services'>, <C72 'Other royalties and license fees'>, <C73 'Purchases and sales of franchises and similar rights'>, <C53 'Insurance Premiums'>, <C71 'Use of franchise and similar rights fees'>, <C48 'Financial'>, <C20 'Legal services'>, <C27 'Other business and professional services'>, <C74 'Purchases and sales of other royalties and licenses'>, <C86 'Other trade in services'>, <C18 'Recruitment '>, <C34 'Computers'>, <C37 'Other information provision services'>, <C65 'Training a

In [8]:
Dimensions = [
            HDim(Year,'Year',DIRECTLY,ABOVE),
            HDim(geo1,'geo1',CLOSEST,ABOVE),
#             HDim(geo2,'geo2',DIRECTLY,LEFT),
            HDim(geo3,'geo3',DIRECTLY,LEFT),
            HDim(Flow, 'Flow',CLOSEST,LEFT),
            HDimConst('Measure Type', 'GBP Total'),
            HDimConst('Unit','gbp-million')
            ]

In [9]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [10]:
new_table = c1.topandas()

In [11]:
new_table['Year'] = pd.to_numeric(new_table['Year'], errors='coerce').fillna(0)

In [12]:
new_table['Year'] = new_table['Year'].astype(int)

In [13]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [14]:
new_table['geo3'].fillna('Total', inplace = True)

In [15]:
new_table['BOP Service'] = new_table['geo1'].fillna('') + '-' + new_table['geo3'].fillna('')

In [16]:
new_table['Flow'] = new_table['Flow'].str.lower()

In [17]:
new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('balances', 'balance'))

In [18]:
new_table = new_table[new_table['Value'] != '' ]

In [19]:
new_table['International Trade Basis'] = 'BOP'

In [20]:
new_table['ONS Partner Geography'] = 'Whole world'

In [21]:
new_table = new_table[['ONS Partner Geography', 'Year','Flow','BOP Service', 'International Trade Basis', 'Measure Type','Value','Unit' ]]

In [22]:
# if is_interactive():
#     SubstancetinationFolder = Path('out')
#     SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
#     new_table.to_csv(SubstancetinationFolder / ('tablec51.csv'), index = False)

In [23]:
new_table.head()

,ONS Partner Geography,Year,Flow,BOP Service,International Trade Basis,Measure Type,Value,Unit
0,Whole world,2009,exports,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,3.852,gbp-million
8,Whole world,2009,balance,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,3.378,gbp-million
13,Whole world,2010,exports,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,95.883,gbp-million
14,Whole world,2011,exports,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,145.03,gbp-million
15,Whole world,2012,exports,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,148.325,gbp-million
